In [1]:
%ls

Alice in Wonderland.pdf
Alice's Adventures in Wonderland by Lewis Carroll.zip
Fairy Tales from Hans Christian Andersen.pdf
Moby Dick; Or, The Whale by Herman Melville.zip
Pride and Prejudice by Jane Austen.zip
README.md
Romeo and Juliet by William Shakespeare.zip
a Little Princess by Frances Hodgson Burnett.zip
crime and punishment.zip
download_from_links.ipynb
game of thrones.csv
lesmisrables.zip
light-novel-titles.csv
lightnovellinks.zip
lotr characters.zip
mobydick.zip
some novels.zip
somelightnovels.zip
war and peace.zip


In [2]:
import requests
from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

In [15]:
switch = True
count = 1
languages = []
authors = []

while switch:
    url = f'https://www.gutenberg.org/ebooks/{count}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extracting language
            try:
                language_row = soup.find('tr', {'property': 'dcterms:language'})
                language = language_row.find('td').text if language_row else '-'
                languages.append(language)  # Corrected variable name
            except Exception as e:
                languages.append('-')  # Append '-' if there's an error
                print(f'Error extracting language for ebook {count}: {e}')
                
            # Extracting author
            try:
                author_name = 'Unavailable'
                for tr in soup.find_all('tr'):
                    th = tr.find('th')
                    print(th)
                    if th and 'Author' in th.text:
                        td = tr.find('td')
                        if td:
                            print(td.text)
                            author_link = td.find('a')
                            if author_link:
                                author_name = author_link.text
                        break
                authors.append(author_name)
                
            except Exception as e:
                authors.append('-')  # Append '-' if there's an error
                print(f'Error extracting author for ebook {count}: {e}')
            count += 1
        else:
            print(f'Failed to fetch data for ebook {count}')
            switch = False
    
    except Exception as e:
        print(f'An error occurred for ebook {count}: {e}')
        count += 1  # Ensure the loop continues even after an error
    if count == 2:  # Adjust this value as per your requirements
        switch = False

# Debug output to verify the results
print()
print()
print()
print(f'Languages: {languages}')
print(f'Authors: {authors}')

<th></th>
None
None
None
None
None
None
None
None
None
<th>Author</th>

Jefferson, Thomas, 1743-1826



Languages: ['English']
Authors: ['Jefferson, Thomas, 1743-1826']


In [90]:
switch = True
count = 12000

book_no = []
titles = []
languages = []
authors = []
origin = []
dates = []

while switch:
    url = f'https://www.gutenberg.org/ebooks/{count}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            book_no.append(count)
            # Extracting title
            try:
                title = soup.find('h1', {'itemprop':'name'}).text
                titles.append(title)
            except Exception as e:
                titles.append('-')  # Append '-' if there's an error
                print(f'Error extracting title for ebook {count}: {e}')
            
            # Extracting language
            try:
                language_row = soup.find('tr', {'property': 'dcterms:language'})
                language = language_row.find('td').text if language_row else '-'
                languages.append(language)  # Corrected variable name
            except Exception as e:
                languages.append('-')  # Append '-' if there's an error
                print(f'Error extracting language for ebook {count}: {e}')
                
            # Extracting author
            try:
                author_name = 'Unavailable'
                for tr in soup.find_all('tr'):
                    th = tr.find('th')
                    # print(th)
                    if th and 'Author' in th.text:
                        td = tr.find('td')
                        # print(td)
                        if td:
                            author_link = td.find('a')
                            if author_link:
                                author_name = author_link.text
                        break
                date_pattern = r', \b\d{4}-\d{4}\b'
                author_name = re.sub(date_pattern, '',author_name)
                authors.append(author_name)
                
            except Exception as e:
                authors.append('-')  # Append '-' if there's an error
                print(f'Error extracting author for ebook {count}: {e}')
            
            # Extracting original publication date
            try:
                for tr in soup.find_all('tr'):
                    th = tr.find('th')
                    pattern = r'[^0-9]'
                    if th and 'Original Publication' in th.text:
                        oridate = tr.find('td').text
                        orid = re.sub(pattern, '', str(oridate))
                        if len(orid) != 4:
                            orid = f'{orid[:4]}'
                            orid += f' - {orid[4:]}'
                        origin.append(orid)
                    else:
                        origin.append('-')
            except Exception as e:
                origin.append('-')  # Append '-' if there's an error
                print(f'Error extracting original publication for ebook {count}: {e}')

            # Extracting release date
            try:
                for tr in soup.find_all('tr'):
                    th = tr.find('th')
                    # pattern = r'[^0-9]'
                    if th and 'Release Date' in th.text:
                        date = tr.find('td', {'itemprop':'datePublished'})
                        date = date.text
                        # if len(date) != 4:
                        #     d = f'{date[:4]}'
                        #     d += f' - {date[4:]}'
                        dates.append(date)
            
            except Exception as e:
                dates.append('-')  # Append '-' if there's an error
                print(f'Error extracting original publication for ebook {count}: {e}')
                
            count += 1
        else:
            print(f'Failed to fetch data for ebook {count}')
            # switch = False
            count += 1
            
        if count % 2000 == 0 or count == 72796:
            # print(book_no)
            book_no_se = pd.Series(book_no)
            titles_se = pd.Series(titles)
            languages_se = pd.Series(languages)
            authors_se = pd.Series(authors)
            origin_se = pd.Series(origin)
            dates_se = pd.Series(dates)
            metadata = pd.DataFrame([book_no_se, titles_se, languages_se, authors_se, origin_se, dates_se]).T
            # metadata = pd.columns=['Book_No', 'Title', 'Language', 'Authors', 'Original Publication', 'Release Date'])
            metadata.to_csv(f'gutenberg_metadata ~{count}.csv', encoding='utf-8', index=False)
            print(f'{count} Done')
        # if count == 100:
        #     print(metadata)
    
    except Exception as e:
        print(f'An error occurred for ebook {count}: {e}')
        count += 1  # Ensure the loop continues even after an error
    if count == 72797:  # Adjust this value as per your requirements
        switch = False
    
        


# # authors = ''

# # for auth in author:
# #     authos = re.sub(date_pattern, '', auth)
# #     if authors != '':
# #         authors += f', {authos}'
# #     else:
# #         authors = authos
# # autho = ', '.join(authors[0].split(',')[:-1]).strip()
# # Debug output to verify the results
# print(f'Book_no: {book_no}')
# print(f'Titles: {titles}')
# print(f'Languages: {languages}')
# print(f'Authors: {authors}')
# print(f'Original Publication: {origin}')
# print(f'Release Date: {dates}')

14000 Done
16000 Done
18000 Done
20000 Done
An error occurred for ebook 21656: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/21656 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x14c347d50>, 'Connection to www.gutenberg.org timed out. (connect timeout=None)'))
22000 Done
24000 Done
26000 Done
28000 Done
Failed to fetch data for ebook 28520
30000 Done
Failed to fetch data for ebook 30360
32000 Done
34000 Done
36000 Done
Failed to fetch data for ebook 36169
38000 Done
40000 Done
An error occurred for ebook 40804: HTTPSConnectionPool(host='www.gutenberg.org', port=443): Max retries exceeded with url: /ebooks/40804 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x14bad4090>, 'Connection to www.gutenberg.org timed out. (connect timeout=None)'))
42000 Done
44000 Done
46000 Done
Failed to fetch data for ebook 47595
48000 Done
50000 Done
52000 Done
54000 Done
56000 Done
Failed to fetc

In [91]:
metadata.to_csv(f'gutenberg_metadata ~{count-1}.csv', encoding='utf-8', index=False)

In [92]:
_12000 = pd.read_csv('./gutenberg_metadata ~12000.csv')
rest = pd.read_csv('./gutenberg_metadata ~72796.csv')

/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_85869/3142092978.py:1: DtypeWarning: Columns (1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  _12000 = pd.read_csv('./gutenberg_metadata ~12000.csv')
/var/folders/0t/816d84452dn7h591gjkg32_m0000gn/T/ipykernel_85869/3142092978.py:2: DtypeWarning: Columns (1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  rest = pd.read_csv('./gutenberg_metadata ~72796.csv')


In [97]:
_12000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295174 entries, 0 to 295173
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       11928 non-null   float64
 1   1       11928 non-null   object 
 2   2       11928 non-null   object 
 3   3       11928 non-null   object 
 4   4       295174 non-null  object 
 5   5       11928 non-null   object 
dtypes: float64(1), object(5)
memory usage: 13.5+ MB


In [98]:
rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498087 entries, 0 to 1498086
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       59987 non-null    float64
 1   1       59987 non-null    object 
 2   2       59987 non-null    object 
 3   3       59987 non-null    object 
 4   4       1498087 non-null  object 
 5   5       59987 non-null    object 
dtypes: float64(1), object(5)
memory usage: 68.6+ MB


In [116]:
metadata = pd.concat([_12000, rest])
metadata = metadata.dropna()
metadata.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 71915 entries, 0 to 59986
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       71915 non-null  float64
 1   1       71915 non-null  object 
 2   2       71915 non-null  object 
 3   3       71915 non-null  object 
 4   4       71915 non-null  object 
 5   5       71915 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.8+ MB


In [117]:
metadata.head(5)

,0,1,2,3,4,5
0,1.0,The Declaration of Independence of the United ...,English,"Jefferson, Thomas",-,"Dec 1, 1971"
1,2.0,The United States Bill of Rights by United States,English,United States,-,"Dec 1, 1972"
2,3.0,John F. Kennedy's Inaugural Address by John F....,English,"Kennedy, John F. (John Fitzgerald)",-,"Nov 1, 1973"
3,4.0,Lincoln's Gettysburg Address by Abraham Lincoln,English,"Lincoln, Abraham",-,"Nov 1, 1973"
4,5.0,The United States Constitution by United States,English,United States,-,"Dec 1, 1975"


In [119]:
# import numpy as np
metadata = metadata.rename(columns={'0':'Book Num', '1':'Book Title', '2':'Language', '3':'Author', '4':'Original Publication Date', '5':'Published Date'})
metadata['Book Num'] = [int(i) for i in metadata['Book Num']]
metadata.head(5)


,Book Num,Book Title,Language,Author,Original Publication Date,Published Date
0,1,The Declaration of Independence of the United ...,English,"Jefferson, Thomas",-,"Dec 1, 1971"
1,2,The United States Bill of Rights by United States,English,United States,-,"Dec 1, 1972"
2,3,John F. Kennedy's Inaugural Address by John F....,English,"Kennedy, John F. (John Fitzgerald)",-,"Nov 1, 1973"
3,4,Lincoln's Gettysburg Address by Abraham Lincoln,English,"Lincoln, Abraham",-,"Nov 1, 1973"
4,5,The United States Constitution by United States,English,United States,-,"Dec 1, 1975"


In [120]:
metadata.to_csv('gutenberg_over_70000_metadata.csv')

In [ ]:
book_no_se = pd.Series(book_no)
titles_se = pd.Series(titles)
languages_se = pd.Series(languages)
authors_se = pd.Series(authors)
origin_se = pd.Series(origin)
dates_se = pd.Series(dates)
metadata = pd.DataFrame([book_no_se, titles_se, languages_se, authors_se, origin_se, dates_se]).T
metadata.rename(columns=['Book_No', 'Title', 'Language', 'Authors', 'Original Publication', 'Release Date'], inplace=True)
metadata.to_csv(f'gutenberg_metadata ~{count}.csv', encoding='utf-8', index=False)

In [32]:
switch = True
count = 88
while switch:
    url = f'https://www.gutenberg.org/cache/epub/{count}/pg{count}.txt'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            text = response.text
            title_line = text.split('\n', 1)[0]
            pattern = '[^a-zA-Z0-9+]'
            title = title_line.replace('The Project Gutenberg eBook of ', '').strip()
            title = re.sub(pattern, '', title)

            # Save the text to a file
            with open(f'./gutenberg_crawled/{count}_{title}.txt', 'w', encoding='utf-8') as file:
                file.write(text)

            # if count == 1:
            #     print(f'First 10 characters of the book: {text[:10]}')
            count += 1
        else:
            print(f'Failed to fetch text for count {count}')
            switch = False

    except Exception as e:
        print(f'An error occurred: {e}')
        continue
    if count == 70000:
        switch = False

Failed to fetch text for count 114
